In [2]:
import pandas as pd
import json
import re
from typing import Dict, List
from dotenv import load_dotenv
from groq import Groq
import os

In [ ]:

# Basic read
with open('raw_inventory.json', 'r', encoding='utf-8') as f:
    Raw_data_json = json.load(f)

with open('taxonomy.json', 'r', encoding='utf-8') as f:
    Ref_data_json = json.load(f)

In [4]:
Raw_data = pd.DataFrame(Raw_data_json)
Raw_data['source_category'].value_counts()

source_category
Front End Loaders          17
4WD Loaders                17
Tracks/Dozer               16
SSL                        15
Crawler Dozers             15
Mini Ex                    13
Earthmoving - Loaders      13
Tractor Loader Backhoe     13
Crawler Excavators         13
Wheel Loaders              12
Utility Tractors           12
Bulldozers                 11
Earthmoving - Dozers       10
Diggers                    10
CTL                         9
TLB                         9
Skid Steers - Track         9
Compact Track Loaders       8
Loader Backhoes             8
Construction/Excavation     7
Hydraulic Excavator         7
Skid Loaders                6
Name: count, dtype: int64

In [5]:
Raw_data['source_category'].unique()

array(['Earthmoving - Dozers', 'Compact Track Loaders',
       'Earthmoving - Loaders', 'Tractor Loader Backhoe', 'SSL',
       'Diggers', 'Skid Steers - Track', 'Front End Loaders',
       'Construction/Excavation', '4WD Loaders', 'Crawler Excavators',
       'Mini Ex', 'CTL', 'Tracks/Dozer', 'Utility Tractors',
       'Wheel Loaders', 'Hydraulic Excavator', 'TLB', 'Bulldozers',
       'Skid Loaders', 'Crawler Dozers', 'Loader Backhoes'], dtype=object)

# Objective - A


In [6]:
MAPPING_FILE = "Testing/Object-A/LLM/category_mapping.json"

if os.path.exists(MAPPING_FILE):
    with open(MAPPING_FILE, "r") as f:
        CATEGORY_MAP = json.load(f)
else:
    CATEGORY_MAP = {}


In [7]:
def normalize_source_category(cat: str) -> str:
    if not cat:
        return ""

    cat = cat.lower().strip()

    cat = cat.replace("-", " ")
    cat = cat.replace("/", " ")
    cat = cat.replace("_", " ")

    cat = " ".join(cat.split())

    return cat


In [8]:

unique_categories = set()

for item in Raw_data_json:
    sc = item.get("source_category")
    if not sc:
        continue
    unique_categories.add(normalize_source_category(sc))

unseen = [
    c for c in unique_categories
    if c not in CATEGORY_MAP
]


In [9]:
with open("taxonomy.json", "r") as f:
    taxonomy = json.load(f)

taxonomy_categories = [t for t in taxonomy]


In [10]:
if unseen:
    prompt = f"""
    You are a heavy equipment classification expert.

    Map each source category to exactly ONE category from the taxonomy list.

    Rules:
    - Use ONLY taxonomy values
    - Return valid JSON
    - One-to-one mapping
    - No explanations

    SOURCE CATEGORIES:
    {unseen}

    TAXONOMY:
    {taxonomy_categories}

    OUTPUT FORMAT:
    {{
    "normalized_source_category": "taxonomy_category"
    }}
    """
    load_dotenv()

    api_key = os.getenv('GROQ_API_KEY')

    client = Groq(api_key=api_key)

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # Fast and smart
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    # mapping = json.loads(response.choices[0].message.content)

    raw = response.choices[0].message.content
    raw = re.sub(r"```json|```", "", raw).strip()

    new_map = json.loads(raw)

    CATEGORY_MAP.update(new_map)

    with open(MAPPING_FILE, "w") as f:
        json.dump(CATEGORY_MAP, f, indent=2)



In [11]:

for item in Raw_data_json:

    norm = normalize_source_category(item.get("source_category", ""))
    item["category"] = CATEGORY_MAP.get(norm, "UNKNOWN")

    # output.append(new_item)


In [12]:

with open("Testing/Object-A/LLM/output_with_llm.json", "w") as f:
    json.dump(Raw_data_json, f, indent=2)


# Objective - B

In [13]:
Raw_data_json[0]

{'serial_number': 'DSX87766',
 'source_category': 'Earthmoving - Dozers',
 'year': 2021,
 'make': 'J.C. Bamford',
 'model': 'D6T',
 'description': 'J.C. BAMFORD D6T - NEEDS UNDERCARRIAGE WORK. - 2WD - 2743 HOURS',
 'category': 'Dozers - Crawler - Medium'}

In [14]:

def extract_cabin(description: str) -> str:
    desc = description.upper()

    # Priority: enclosed beats open
    if "ENCLOSED CAB" in desc or "EROPS" in desc:
        return "EROPS"
    if "OPEN STATION" in desc or "OROPS" in desc:
        return "OROPS"
    return "Unknown"



def extract_drive(description: str) -> str:
    desc = description.upper()

    if "TRACK" in desc:
        return "Tracks"
    if "AWD" in desc:
        return "AWD"
    if "4WD" in desc or "4X4" in desc:
        return "4WD"
    if "2WD" in desc:
        return "Wheels"

    return "Unknown"



def extract_hours(description: str):
    desc = description.upper()

    match = re.search(
        r'(?:HRS|HOURS|METER READS:?)\s*[:\-]?\s*(\d{1,3}(?:,\d{3})+|\d+)|(\d{1,3}(?:,\d{3})+|\d+)\s*(?:HRS|HOURS)',
        desc
    )

    if match:
        value = match.group(1) or match.group(2)
        return int(value.replace(",", ""))

    return None

def extract_features(record):
    desc = record.get("description", "")

    record["cabin"] = extract_cabin(desc)
    record["drive"] = extract_drive(desc)
    record["hours"] = extract_hours(desc)

    return record


In [15]:
for i, record in enumerate(Raw_data_json):
    features = extract_features(record)
    Raw_data_json[i].update(features)


with open("Testing/Object-B/output_with_re.json", "w") as f:
    json.dump(Raw_data_json, f, indent=2)


# Objective - C

In [21]:
raw_cat_data = pd.DataFrame(Raw_data_json)

In [22]:
raw_cat_data

,serial_number,source_category,year,make,model,description,category,cabin,drive,hours
0,DSX87766,Earthmoving - Dozers,2021,J.C. Bamford,D6T,J.C. BAMFORD D6T - NEEDS UNDERCARRIAGE WORK. -...,Dozers - Crawler - Medium,Unknown,Wheels,2743
1,KFC16572,Compact Track Loaders,2016,Bobcat,259D,2016 BOBCAT 259D - JOBSITE READY. - GP BUCKET ...,Skid Steer Loaders - Track,EROPS,Unknown,294
2,AWH92122,Earthmoving - Loaders,2022,Caterpillar Inc.,906M,2022 CATERPILLAR INC. 906M - RUNS AND DRIVES G...,Wheel Loaders - Large,Unknown,4WD,6883
3,KMK76627,Tractor Loader Backhoe,2023,J. Deere,310SL,2023 J. DEERE 310SL - JOBSITE READY. - 4X4 - E...,Backhoe Loaders,EROPS,4WD,3135
4,ITE11805,Earthmoving - Loaders,2022,Kubota Tractor Corp,938M,"KUBOTA TRACTOR CORP 938M - NICE UNIT, READY TO...",Wheel Loaders - Large,Unknown,AWD,3708
...,...,...,...,...,...,...,...,...,...,...
245,BIC21763,Earthmoving - Dozers,2015,Caterpillar Inc.,D6K,2015 CATERPILLAR INC. D6K - WELL MAINTAINED EX...,Dozers - Crawler - Medium,OROPS,Unknown,4537
246,XXD50818,Earthmoving - Loaders,2019,J.C. Bamford,544K,"J.C. BAMFORD 544K - CLEAN CAB, COLD AC. - HYDR...",Wheel Loaders - Large,EROPS,Unknown,673
247,VUI78089,Bulldozers,2022,Kubota Tractor Corp,D6K,KUBOTA TRACTOR CORP D6K - EX-RENTAL FLEET. - C...,Dozers - Crawler - Large,Unknown,Unknown,6659
248,SGU53553,4WD Loaders,2024,Volvo Construction Equipment,938M,2024 VOLVO CONSTRUCTION EQUIPMENT 938M - NEEDS...,Wheel Loaders - Compact,EROPS,Wheels,6371


In [23]:
raw_cat_data['make'].unique()

array(['J.C. Bamford', 'Bobcat', 'Caterpillar Inc.', 'J. Deere',
       'Kubota Tractor Corp', 'Volvo CE', 'Deere', 'John Deere',
       'Komatsu America', 'Caterpillar', 'JCB Construction', 'Kubota',
       'Bobcat Co.', 'Hitachi Construction Machinery', 'Bobcat Company',
       'Cat', 'CAT', 'Volvo', 'Volvo Construction Equipment', 'JCB',
       'Hitachi', 'Komatsu', 'Komatsu Ltd', 'Deere & Co.'], dtype=object)

## APPROACH - 1 Using Defined Map

In [115]:
MANUFACTURER_MAP = {
    # Caterpillar
    "CAT": "Caterpillar",
    "Cat": "Caterpillar",
    "Caterpillar": "Caterpillar",
    "Caterpillar Inc.": "Caterpillar",

    # John Deere
    "Deere": "John Deere",
    "J. Deere": "John Deere",
    "John Deere": "John Deere",
    "Deere & Co.": "John Deere",

    # Bobcat
    "Bobcat": "Bobcat",
    "Bobcat Co.": "Bobcat",
    "Bobcat Company": "Bobcat",

    # Komatsu
    "Komatsu": "Komatsu",
    "Komatsu Ltd": "Komatsu",
    "Komatsu America": "Komatsu",

    # Volvo
    "Volvo": "Volvo",
    "Volvo CE": "Volvo",
    "Volvo Construction Equipment": "Volvo",

    # Hitachi
    "Hitachi": "Hitachi",
    "Hitachi Construction Machinery": "Hitachi",

    # Kubota
    "Kubota": "Kubota",
    "Kubota Tractor Corp": "Kubota",

    # JCB
    "JCB": "JCB",
    "JCB Construction": "JCB",

    # J.C. Bamford (JCB parent)
    "J.C. Bamford": "JCB",
}


In [116]:
def normalize_manufacturer(make: str) -> str:
    if not isinstance(make, str):
        return "Unknown"

    make = make.strip()

    return MANUFACTURER_MAP.get(make, make)
raw_cat_data["make_normalized"] = raw_cat_data["make"].apply(normalize_manufacturer)
raw_cat_data[["make", "make_normalized"]].drop_duplicates().sort_values("make")


,make,make_normalized
1,Bobcat,Bobcat
16,Bobcat Co.,Bobcat
21,Bobcat Company,Bobcat
27,CAT,Caterpillar
26,Cat,Caterpillar
12,Caterpillar,Caterpillar
2,Caterpillar Inc.,Caterpillar
8,Deere,John Deere
72,Deere & Co.,John Deere
44,Hitachi,Hitachi


## APPROACH - 2 LLM Based

In [16]:
import re
import json
import os


In [27]:
def normalize_key(s):
    if not isinstance(s, str):
        return ""
    s = s.lower().strip()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s)
    return s


def parse_llm_json(text):
    # 1. Extract JSON block
    match = re.search(r"\{[\s\S]*\}", text)
    if not match:
        raise ValueError("No JSON object found")

    raw = match.group()

    # 2. Fix common LLM issues
    # smart quotes → normal quotes
    raw = raw.replace("“", '"').replace("”", '"')
    raw = raw.replace("‘", "'").replace("’", "'")
    raw = raw.replace("\u00a0", " ")

    # 3. Remove illegal JSON escape: \'
    raw = re.sub(r"\\'", "'", raw)

    # 4. Convert remaining single quotes to double quotes
    raw = raw.replace("'", '"')

    # 5. Final parse
    return json.loads(raw)


In [28]:
MAPPING_FILE = "Testing/Object-C/manufacturer_mapping.json"
OUTPUT_FILE = "Testing/Object-C/raw_inventory_normalized.json"

if os.path.exists(MAPPING_FILE):
    with open(MAPPING_FILE, "r") as f:
        MANUFACTURER_MAP = json.load(f)
else:
    MANUFACTURER_MAP = {}

In [29]:
raw_makes = set(
    item["make"] for item in Raw_data_json
    if isinstance(item.get("make"), str)
)


In [30]:
unseen = []
for m in raw_makes:
    if normalize_key(m) not in MANUFACTURER_MAP:
        unseen.append(m)


In [31]:
if unseen:
    client = Groq(api_key=os.getenv("GROQ_API_KEY"))

    prompt = f"""
Normalize heavy equipment manufacturer names.

Return ONLY JSON:
{{ "<input>": "<canonical>" }}

Examples:
{{ "CAT": "Caterpillar", "J. Deere": "John Deere" }}

Rules:
- Expand abbreviations
- Remove Inc, Corp, Ltd
- Use construction equipment context
- Use double quotes (") only. Do not use single quotes it is a very strick rule.


Input:
{json.dumps(unseen)}
"""

    resp = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    llm_map = parse_llm_json(resp.choices[0].message.content)

    # save to cache
    for raw, canon in llm_map.items():
        MANUFACTURER_MAP[normalize_key(raw)] = canon

    with open(MAPPING_FILE, "w") as f:
        json.dump(MANUFACTURER_MAP, f, indent=2)






In [33]:
# ------------------
# apply normalization
# ------------------

for item in Raw_data_json:
    make = item.get("make")
    if isinstance(make, str):
        item["make"] = MANUFACTURER_MAP.get(
            normalize_key(make),
            make
        )

# Test on new Data

In [62]:

if os.path.exists(MAPPING_FILE):
    with open(MAPPING_FILE, "r") as f:
        mapping = json.load(f)
else:
    mapping = {}



In [63]:
test_makes = [
    "CAT",                # likely already known
    "Caterpillar Inc.",   # variant
    "J. Deere",           # variant
    "Deere & Co",         # variant
    "Kubota Corp",        # maybe known
    "Takeuchi Mfg.",      # usually unseen
    "SANY Heavy Ind",     # unseen
    "XCMG Group",         # unseen
    "Doosan Infracore",   # maybe unseen
    "Bob-Cat Co"          # tricky spelling
]
inventory = []

for i, make in enumerate(test_makes, start=1):
    inventory.append({
        "serial_number": f"TEST-{i:04d}",
        "make": make,
        "model": f"MODEL-{i}",
        "description": f"Test machine from {make}"
    })
with open("Testing/Object-C/Testing.json", "w") as f:
    json.dump(inventory, f, indent=2)

In [64]:
with open('Testing/Object-C/Testing.json', 'r', encoding='utf-8') as f:
    Test_data = json.load(f)

In [65]:
raw_makes_test = set(
    item["make"] for item in Test_data
    if isinstance(item.get("make"), str)
)
raw_makes_test

{'Bob-Cat Co',
 'CAT',
 'Caterpillar Inc.',
 'Deere & Co',
 'Doosan Infracore',
 'J. Deere',
 'Kubota Corp',
 'SANY Heavy Ind',
 'Takeuchi Mfg.',
 'XCMG Group'}

In [66]:
unseen = []
for m in raw_makes_test:
    if normalize_key(m) not in mapping:
        unseen.append(m)
unseen

['Takeuchi Mfg.',
 'SANY Heavy Ind',
 'Doosan Infracore',
 'XCMG Group',
 'Kubota Corp']

In [67]:
if unseen:
    client = Groq(api_key=os.getenv("GROQ_API_KEY"))

    prompt = f"""
Normalize heavy equipment manufacturer names.

Return ONLY JSON:
{{ "<input>": "<canonical>" }}

Examples:
{{ "CAT": "Caterpillar", "J. Deere": "John Deere" }}

Rules:
- Expand abbreviations
- Remove Inc, Corp, Ltd
- Use construction equipment context

Input:
{json.dumps(unseen)}
"""

    resp = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    llm_map = parse_llm_json(resp.choices[0].message.content)

    # save to cache
    for raw, canon in llm_map.items():
        MANUFACTURER_MAP[normalize_key(raw)] = canon

    with open(MAPPING_FILE, "w") as f:
        json.dump(MANUFACTURER_MAP, f, indent=2)

# 



## Objective - D


In [35]:
Raw_data_json[0]

{'serial_number': 'DSX87766',
 'source_category': 'Earthmoving - Dozers',
 'year': 2021,
 'make': 'JCB',
 'model': 'D6T',
 'description': 'J.C. BAMFORD D6T - NEEDS UNDERCARRIAGE WORK. - 2WD - 2743 HOURS',
 'category': 'Dozers - Crawler - Medium',
 'cabin': 'Unknown',
 'drive': 'Wheels',
 'hours': 2743}

In [36]:
processed_inventory = []

for record in Raw_data_json:
    processed_inventory.append({
        "serial_number": record.get("serial_number"),
        "description": record.get("description"),
        "category": record.get("category"),
        "make": record.get("make"),
        "model": record.get("model"),
        "extracted_features": {
            "cabin": record.get("cabin", "Unknown"),
            "drive": record.get("drive", "Unknown"),
            "hours": record.get("hours")
        }
    })


In [37]:
import json

with open("processed_inventory.json", "w") as f:
    json.dump(processed_inventory, f, indent=2)
